In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import tensorflow as tf
import os, math, cv2, glob, random, time
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
from keras.optimizers import SGD, Adam
from keras.callbacks import LearningRateScheduler
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Reshape, Flatten, concatenate, BatchNormalization, Conv2DTranspose, LeakyReLU, Dropout


train_dataset_path = 'training/training'
validation_dataset_path  = 'validation/validation'

# width 550 height 367
IMAGE_HEIGHT      = 64
IMAGE_WIDTH       = 64
IMAGE_SIZE        = (IMAGE_HEIGHT, IMAGE_WIDTH)

CATEGORIES        = os.listdir(train_dataset_path)
# len all images 1370
ALL_DATA_LENGTH   = 1370
INPUT_DATA_LENGTH = 100
INPUT_DENSE       = IMAGE_HEIGHT * IMAGE_WIDTH * 3
BATCH_SIZE        = 32 
tf.__version__

'2.1.0'

In [3]:
x= np.random.choice(np.arange(INPUT_DATA_LENGTH), INPUT_DATA_LENGTH)

def get_image_to_path():
    idxIn = 0; namesIn = [];
    imagesIn = np.zeros((ALL_DATA_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3))
    for path_im in [train_dataset_path, validation_dataset_path]:
        for category in CATEGORIES:
            path = f'{path_im}/{category}/'
            class_id = CATEGORIES.index(category)
            for image in os.listdir(path):
                img = Image.open(os.path.join(path, image))
                w = img.size[0]
                h = img.size[1]
                sz = np.min((w,h))
                a=0; b=0
                if w<h: b = (h-sz)//2
                else: a = (w-sz)//2
                img = img.crop((0+a, 0+b, sz+a, sz+b))  
                img = img.resize((IMAGE_HEIGHT, IMAGE_WIDTH), Image.ANTIALIAS)   
                imagesIn[idxIn,:,:,:] = np.asarray(img)
                namesIn.append(os.path.join(path, image))
                idxIn += 1
    idx = np.arange(idxIn)
    np.random.shuffle(idx)
    imagesIn = imagesIn[idx,:,:,:]
    namesIn = np.array(namesIn)[idx]
    return imagesIn, namesIn
    
data_images, data_names = get_image_to_path()

In [4]:
# BUILD DISCRIMINATIVE NETWORK
monkey = Input((INPUT_DENSE,))
monkeyName = Input((INPUT_DATA_LENGTH,))
x = Dense(INPUT_DENSE, activation='sigmoid')(monkeyName) 
x = Reshape((2,INPUT_DENSE,1))(concatenate([monkey,x]))
x = Conv2D(1,(2,1),use_bias=False,name='conv')(x)
discriminated = Flatten()(x)

# COMPILE
discriminator = Model([monkey,monkeyName], discriminated)
discriminator.get_layer('conv').trainable = False
discriminator.get_layer('conv').set_weights([np.array([[[[-1.0 ]]],[[[1.0]]]])])
discriminator.compile(optimizer='adam', loss='binary_crossentropy')

# DISPLAY ARCHITECTURE
discriminator.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 12288)        0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 12288)        1241088     input_2[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 24576)        0           input_1[0][0]                    
                                                                 dense_3[0][0]              

In [5]:
# TRAINING DATA
train_y = (data_images[:INPUT_DATA_LENGTH,:,:,:]/255.).reshape((-1,INPUT_DENSE))
train_X = np.zeros((INPUT_DATA_LENGTH,INPUT_DATA_LENGTH))
for i in range(INPUT_DATA_LENGTH): train_X[i,i] = 1
zeros = np.zeros((INPUT_DATA_LENGTH,INPUT_DENSE))

# TRAIN NETWORK
lr = 0.5
for k in range(5):
    annealer = LearningRateScheduler(lambda x: lr)
    h = discriminator.fit([zeros,train_X],
                          train_y,
                          epochs=5,
                          batch_size=BATCH_SIZE,
                          callbacks=[annealer],
                          verbose=0)
    print('Epoch',(k+1)*10,'/30 - loss =',h.history['loss'][-1] )
    if h.history['loss'][-1]<0.533: lr = 0.1

InternalError:  cuDNN launch failure : input shape([32,1,2,12288]) filter shape([2,1,1,1])
	 [[node conv/convolution (defined at /home/zahid/anaconda3/envs/work_env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_1955]

Function call stack:
keras_scratch_graph
